In [1]:
import pandas as pd
import numpy as np
import openpyxl
import random

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier

df = pd.read_csv("atp_tennis.csv")


objs = df.select_dtypes("object").columns
print(", ".join(objs))

Tournament, Date, Series, Court, Surface, Round, Player_1, Player_2, Winner, Score


In [ ]:
df["player1Elo"] = 0.0
df["player2Elo"] = 0.0
df["player1CourtProp"] = 0.0
df["player2CourtProp"] = 0.0
